In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 14


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
4,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-10-05T22:58:24-05:00 -> 1696564704000, Until: 2023-10-06T22:58:24-05:00 -> 1696651104000
Group by Time: 30s


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp', GROUP_BY_TIME)
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,
2023-10-05 22:58:30-05:00,-147.2,28.47,27.49,NaN,NaN,25.78,29.13,NaN,28.57,27.30,27.0,25.9,29.92,26.73
2023-10-05 22:59:00-05:00,-147.2,28.47,27.50,NaN,NaN,25.74,29.13,NaN,28.56,27.33,27.0,25.8,29.92,26.76
2023-10-05 22:59:30-05:00,-147.2,28.47,27.50,NaN,NaN,25.71,29.12,NaN,28.54,27.33,27.0,25.9,29.89,26.73
2023-10-05 23:00:00-05:00,-147.2,28.48,27.47,NaN,NaN,25.72,29.15,NaN,28.53,27.35,27.0,25.8,29.90,26.76
2023-10-05 23:00:30-05:00,-147.2,28.50,27.49,NaN,NaN,25.71,29.12,NaN,28.51,27.37,27.0,25.8,29.87,26.76


## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,2.708000e+03,2619.000000,2666.000000,1303.000000,2552.000000,2333.000000,2579.000000,2535.000000,2647.000000,2666.000000,2602.000000,2697.000000,2693.000000,2682.000000
mean,-1.472000e+02,29.318160,29.147479,33.208012,28.478801,28.233480,31.445936,30.086686,30.945890,28.036804,29.470792,27.879607,30.445273,29.784183
std,5.685392e-14,2.424353,4.047249,2.911822,2.303600,2.676031,2.957641,3.916063,4.228796,2.504207,3.514274,3.205445,2.654537,3.308571
min,-1.472000e+02,25.460000,23.850000,28.930000,25.670000,24.820000,28.380000,25.470000,26.520000,24.550000,24.700000,23.800000,26.590000,25.670000
25%,-1.472000e+02,26.880000,25.492500,30.670000,26.010000,25.760000,28.820000,26.280000,27.240000,26.130000,26.100000,25.100000,27.710000,26.505000
50%,-1.472000e+02,29.700000,28.550000,32.450000,28.595000,28.280000,30.310000,29.500000,29.290000,27.180000,29.400000,27.400000,30.070000,29.785000
75%,-1.472000e+02,30.890000,31.450000,36.365000,30.255000,29.380000,33.780000,33.400000,34.630000,30.600000,31.800000,30.300000,32.360000,32.810000
max,-1.472000e+02,34.090000,37.700000,38.120000,33.040000,36.920000,37.840000,37.620000,39.150000,32.630000,36.400000,34.400000,35.810000,35.450000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 45%, Missing: 55%, To be checked
Tangara Sensor: TANGARA_1EE6, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 81%, Missing: 19%, To be checked
Tangara Sensor: TANGARA_2FF6, Data: 88%, Missing: 12%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")